### Load GEE Satellite Data
Example code of downloading data from the GEE Community Catalog and the Earth Engine Data Catalog. Requires the use of the `rabpro` package. Additionally, the `ee` python package has to be configured to access your Google Earth Engine account prior to running the lines in this notebook. Replace `[GEE-PROJECT-NAME]` with the name of your project in the code editor.

Climate/Anthropogenic Data:
- [Rangeland Analysis Platform Layers](https://gee-community-catalog.org/projects/rap/)
- [Land Change Monitoring, Assessment, and Projection (LCMAP)](https://gee-community-catalog.org/projects/lcmap/)
- [JRC Monthly Water Recurrence](https://developers.google.com/earth-engine/datasets/catalog/JRC_GSW1_4_MonthlyRecurrence)

The polygon geometries of the study basins are uploaded to the Google Earth Engine code editor as a shapefile.

In [ ]:
import geopandas as gpd

subbasins = gpd.read_file('Data/basins/attributes.gpkg')
subbasins.geometry = subbasins.geometry.simplify(0.01)
subbasins.to_file('Data/basins/attributes.shp')

In [ ]:
import ee
import os

os.environ['http_proxy'] = 'http://proxyout.lanl.gov:8080'
os.environ['https_proxy'] = 'http://proxyout.lanl.gov:8080'

ee.Initialize()

In [ ]:
import rabpro
from rabpro.basin_stats import Dataset
from rabpro import utils as ru
from rabpro import data_utils as du
import geopandas as gpd

In [ ]:
# GEE asset polygons
project_name = '[GEE-PROJECT-NAME]'
asset = 'projects/'+project_name+'/assets/vote_basins'

# output .csvs folder
gdrive_folder_name = 'vote_basins'

# rabpro data dictionary
hpu_datasets = {
    'land_cover' : {'path' : 'projects/sat-io/open-datasets/LCMAP/LCPRI',
                    'band' : 'b1',
                    'stats' : ['freqhist'],
                    'start': '1985-01-01',
                    'end' : '2021-01-01'},
    'veg_cover' : {'path' : 'projects/rap-data-365417/assets/vegetation-cover-v3',
                 'band' : 'None',
                 'stats' : ['freqhist'],
                 'start' : '1986-01-01',
                 'end' : '2021-01-01'}
    'monthly_recurrence' : {'path' : 'JRC/GSW1_4/MonthlyRecurrence',
                            'band' : 'monthly_recurrence',
                            'stats' : ['sum'],
                            'start': '1985-01-01',
                            'end' : '2021-12-31'},
    }

In [ ]:
dataset_list = []
for key in hpu_datasets:
    this_ds = hpu_datasets[key]
    mosaic, start, end = False, None, None
    if this_ds == 'custom':
        continue
    if 'mosaic' in this_ds.keys() and this_ds['mosaic'] is True:
        mosaic = True
    if 'start' in this_ds.keys():
        start = this_ds['start']
    if 'end' in this_ds.keys():
        end = this_ds['end']
        
    dataset_list.append(Dataset(this_ds['path'], this_ds['band'],
#                                 this_ds['start'], this_ds['end'],
                                stats=this_ds['stats'], mosaic=mosaic))
    
# access datasets
urls, tasks = rabpro.basin_stats.compute(dataset_list,
                           gee_feature_path=asset,
                           folder=gdrive_folder_name,
                                        validate_dataset_list=False)

After executing these lines, move the data from your Google Drive folder to the following directory: `Data/gee/`.